## Benchmark of routines for generation of weak integer compositions (ordered integer partitions with lower bound 0)

In [1]:
using BenchmarkTools
using Combinatorics:
    partitions,
    permutations,
    WithReplacementCombinations,
    with_replacement_combinations
using IterTools:
    Iterators.popfirst!,
    Iterators.Stateful

In [2]:
######################
# New implementation #
######################

# NOTE: slightly slower and ~20% more allocs this way, but probably
#       more Julian, and doesn't modify Combinatorics source code

struct WeakIntegerCompositions{T}
    it::WithReplacementCombinations{T}
end

Base.eltype(::Type{WeakIntegerCompositions{T}}) where {T} = Vector{eltype(T)}
Base.length(c::WeakIntegerCompositions) = length(c.it)

"""
Generate all weak compositions of an integer `n` with size `t`.
"""
weak_integer_compositions(n::Integer, t::Integer) = WeakIntegerCompositions(
    WithReplacementCombinations(1:t, n)
)

function Base.iterate(c::WeakIntegerCompositions, s=Stateful(c.it))
    # Advance stateful version of member iterator
    isempty(s) && return
    # Reinterpret size n multicombinations (combinations with replacement) of [1:t]
    # as size n weak integer compositions of t by histogramming indices
    weak_comp = zeros(Int, length(c.it.a))
    # for i in this_multicombination
    for i in popfirst!(s)
        weak_comp[i] += 1
    end
    (weak_comp, s)
end

################################
# Alternate new implementation #
################################

# NOTE: This is modified Combinatorics.jl source code! Duplicate of WithReplacementCombinations
#       iterator with additional post-processing step to reinterpret as weak compositions

struct WeakIntegerCompositionsV2{T}
    a::T
    t::Int
end

Base.eltype(::Type{WeakIntegerCompositionsV2{T}}) where {T} = Vector{eltype(T)}
Base.length(c::WeakIntegerCompositionsV2) = binomial(length(c.a) + c.t - 1, c.t)

"""
Generate all weak compositions of an integer `n` with size `t`.
"""
weak_integer_compositions_v2(n::Integer, t::Integer) = WeakIntegerCompositionsV2(1:t, n)

function Base.iterate(c::WeakIntegerCompositionsV2, s=[1 for i in 1:c.t])
    (!isempty(s) && s[1] > length(c.a) || c.t < 0) && return
    n = length(c.a)
    t = c.t
    comb = [c.a[si] for si in s]
    if t > 0
        s = copy(s)
        changed = false
        for i in t:-1:1
            if s[i] < n
                s[i] += 1
                for j in (i+1):t
                    s[j] = s[i]
                end
                changed = true
                break
            end
        end
        !changed && (s[1] = n + 1)
    else
        s = [n + 1]
    end
    # Reinterpret size t multicombinations (combinations with replacement) of [1:n]
    # as size n weak integer compositions of t by histogramming indices
    weak_comp = zeros(Int, n)
    for i in comb
        weak_comp[i] += 1
    end
    (weak_comp, s)
end

########################
# Kun's implementation #
########################

# NOTE: This is function `Parquet.orderedPartition`, specialized to weak
#       compositions (lowerbound = 0) and without debug assertions

"""
Generate all weak compositions of an integer `n` with size `t`.
"""
function weak_integer_compositions_kun(t, n)
    unorderedPartition = collect(partitions(t + n, n))
    orderedPartition = Vector{Vector{Int}}([])
    for p in unorderedPartition
        p = p .- 1
        append!(orderedPartition, Set(permutations(p)))
    end
    return orderedPartition
end

weak_integer_compositions_kun

In [3]:
loop_num = 5;
n_expandables = 5;

In [4]:
sort(collect(weak_integer_compositions(5, 5)))

126-element Vector{Vector{Int64}}:
 [0, 0, 0, 0, 5]
 [0, 0, 0, 1, 4]
 [0, 0, 0, 2, 3]
 [0, 0, 0, 3, 2]
 [0, 0, 0, 4, 1]
 [0, 0, 0, 5, 0]
 [0, 0, 1, 0, 4]
 [0, 0, 1, 1, 3]
 [0, 0, 1, 2, 2]
 [0, 0, 1, 3, 1]
 ⋮
 [3, 1, 0, 0, 1]
 [3, 1, 0, 1, 0]
 [3, 1, 1, 0, 0]
 [3, 2, 0, 0, 0]
 [4, 0, 0, 0, 1]
 [4, 0, 0, 1, 0]
 [4, 0, 1, 0, 0]
 [4, 1, 0, 0, 0]
 [5, 0, 0, 0, 0]

In [5]:
# Verify that all three implementations give equivalent results
@assert allequal([sort(collect(v)) for v in [weak_integer_compositions(loop_num, n_expandables), 
                                             weak_integer_compositions_v2(loop_num, n_expandables), 
                                             weak_integer_compositions_kun(loop_num, n_expandables)]])

### Case 1: Iterator carrying a `Stateful` instance of `Combinatorics.WithReplacementCombinations`

In [9]:
b1 = @benchmarkable collect(weak_integer_compositions(loop_num, n_expandables)) samples=100000 evals=10
run(b1)

BenchmarkTools.Trial: 31018 samples with 10 evaluations.
 Range (min … max):  12.755 μs … 380.865 μs  ┊ GC (min … max):  0.00% … 95.81%
 Time  (median):     13.585 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   15.874 μs ±  21.874 μs  ┊ GC (mean ± σ):  13.73% ±  9.32%

    ▂█▆▂                                                        
  ▂▅████▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▃
  12.8 μs         Histogram: frequency by time         24.7 μs <

 Memory estimate: 44.55 KiB, allocs estimate: 634.

### Case 2: Direct iterator implementation (using modified `Combinatorics.jl` source code for `WithReplacementCombinations`)

In [10]:
b2 = @benchmarkable collect(weak_integer_compositions_v2(loop_num, n_expandables)) samples=100000 evals=10
run(b2)

BenchmarkTools.Trial: 31749 samples with 10 evaluations.
 Range (min … max):  12.856 μs … 331.682 μs  ┊ GC (min … max):  0.00% … 93.53%
 Time  (median):     13.711 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   15.514 μs ±  16.734 μs  ┊ GC (mean ± σ):  10.22% ±  8.76%

   ▃▅▇█▇▅▄▃▂▂▁▁▁▁▁                                             ▂
  ██████████████████▇▇▇▆▆▅▇▆▆▆▆▆▆▇▆▆▆▆▆▆▆▅▆▇▇▆▃▅▅▄▅▅▅▅▆▆▅▆▆▅▅▅ █
  12.9 μs       Histogram: log(frequency) by time      23.6 μs <

 Memory estimate: 40.56 KiB, allocs estimate: 507.

### Case 3: Existing implementation, i.e., `Parquet.orderedPartition` stripped of assertions and specialized to weak compositions (`lowerbound=0`)

In [11]:
b3 = @benchmarkable collect(weak_integer_compositions_kun(loop_num, n_expandables)) samples=100000 evals=10
run(b3)

BenchmarkTools.Trial: 4766 samples with 10 evaluations.
 Range (min … max):   86.960 μs … 459.511 μs  ┊ GC (min … max): 0.00% … 73.00%
 Time  (median):      93.038 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   104.247 μs ±  46.291 μs  ┊ GC (mean ± σ):  9.31% ± 14.22%

  ▄█▅▁                                                          ▁
  ████▇▇▅▅▆▁▆▆▆▆▆▇▄▄▁▄▃▃▄▁▃▃▄▁▁▄▁▄▃▃▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▇▇████▇ █
  87 μs         Histogram: log(frequency) by time        314 μs <

 Memory estimate: 207.95 KiB, allocs estimate: 2599.

We observe that both iterators outperform the existing implementation in terms of runtime and memory allocations, especially in the regime $n_l < n_e$ where $n_l$ is the loop number and $n_e$ is the number of propagators to be expanded. This is because the simple implementation first generates all permutations for each cycle, then identifies and discards duplicates. In other words, the algorithm does not take advantage of the multiset structure.

In contrast, the Combinatorics.jl iterator `with_replacement_combinations` directly generates multicombinations of fixed size. For instance, consider the following two cases: (1) $n_l = 2, n_e = 5$, and (2) $n_l = 5, n_e = 10$.
The naive approach is an order of magnitude slower for case (1), and completely intractable in case (2). Even at high orders as in case (2), the more sophisticated multicombination iterator uses less than 1 MB of memory and retains a runtime of under a millisecond. 

We propose to use the first of the two iterator implementations; it is slightly less efficient than the direct implementation, but it is more Julian, and also avoids any licensing headaches associated with modifying the `Combinatorics.jl` sources. Note that this would currently only replace the functionality of `Parquet.orderedPartition` for the generation of *weak* compositions, i.e., when `lowerbound=0`.

Finally, note that many complicated optimizations exist for (multi)permutation/combination generation (Gray code, Co-lex and Cool-lex orders, etc.)—I have not reverse-engineered the actual implementation in `Combinatorics.jl`, but it would be interesting to do so.